In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
import time
import matplotlib.pyplot as plt
import math

### Data Collection ###

__Scrape using the GWAS Catalog's API__

In [2]:
gwas_catalog = pd.read_json("https://www.ebi.ac.uk/gwas/summary-statistics/api/traits/EFO_0003785/associations")

# Subset the list of associations
associations = gwas_catalog["_embedded"]["associations"]

scraped_df = pd.DataFrame(columns = ["BETA", "CHR", "BP", "OR", "RAF", "PVALUE", "VARIANT", "EFFECT_ALLELE", 
                                     "TRAIT", "MAPPED_GENE_LIST", "NUM_MAPPED_GENES", "TRAITS_ASSOCIATED_WITH_MAPPED_GENES"])

for i in range(len(associations)):
    obj = associations[str(i)] # association object
    
    # Obtain all the info I need
    beta = obj["beta"]
    chrom = obj["chromosome"]
    bp = obj["base_pair_location"]
    odd_rat = obj["odds_ratio"]
    eff_allele_freq = obj["effect_allele_frequency"]
    p_val = obj["p_value"]
    variant = obj["variant_id"]
    eff_allele = obj["effect_allele"]
    trait = "Atopy"
    
    # Create new row and add it to the dataframe
    scraped_df.loc[len(scraped_df)] = [beta, chrom, bp, odd_rat, eff_allele_freq, p_val, variant, eff_allele, trait, [], 0, "-"]

__Read all the data from the csv files and add it all to one dataframe__

In [3]:
trait_files = ["all-rhin-associations-2020-09-27.csv", "all-allergy-associations-2020-09-28.csv"]
additional_trait_codes = ["0000274", "0002686", "0003956", "0005298", "0007016", 
                          "0007017", "0007018", "0007019", "1000651", "1001243"]

for val in additional_trait_codes:
    new_file = "efotraits_EFO_" + val + "-associations-2020-09-30.csv" # assemble file name
    trait_files.append(new_file)

# Create a dataframe will the data from all the csv files
all_traits = pd.read_csv(trait_files[0])
for file in trait_files[1:]:
    new_trait = pd.read_csv(file)
    all_traits = all_traits.append(new_trait, ignore_index = True)

First, I split the variant and risk allele column into two columns so that I can process both separately. When loading the original column as a dataframe the HTML b tag was read as a string so I used the string version of the tag as a delimeter.

In [4]:
all_traits["VARIANT"] = [i.split("-<b>")[0] for i in all_traits["Variant and risk allele"]]
all_traits["EFFECT_ALLELE"] = [i.split("-<b>")[1][0] for i in all_traits["Variant and risk allele"]]

Then, I split the location column into two columns (chromosome and base pair location). This time I used a colon as the delimeter.

In [5]:
all_traits["CHR"] = [i.split(":")[0] for i in all_traits["Location"]]
all_traits["BP"] = [i.split(":")[1] if len(i.split(":")) > 1 else i.split(":")[0] for i in all_traits["Location"]]

Next, I converted the mapped gene column to a list and store the number of mapped genes

In [6]:
all_traits["MAPPED_GENE_LIST"] = [ i.split(", ") for i in all_traits["Mapped gene"] ]
all_traits["NUM_MAPPED_GENES"] = [ len(i.split(", ")) for i in all_traits["Mapped gene"] ]

Then, I added another column that contains traits associated with the same mapped genes.

In [7]:
food_allergy = pd.read_csv("food allergy-DOID_3044-genes-2020-09-27.tsv", sep = "\t")

gene_disease_dict = {}
for val in food_allergy[["Gene Symbol", "Disease Name"]].itertuples():
    gene_disease_dict[val[1]] = val[2]

prev_associated_traits = []
for gene_list in all_traits["MAPPED_GENE_LIST"]:
    prev_trait_list = [] # use a list in case there are multiple previous associations
    for gene in gene_disease_dict:
        if gene in gene_list:
            prev_trait_list.append(gene_disease_dict[gene]) # add the previously associated disease
            
    # Represent no previous associations using a hypen instead of an empty list
    if prev_trait_list == []:
        prev_trait_list = "-"
    
    prev_associated_traits.append(prev_trait_list)

all_traits["TRAITS_ASSOCIATED_WITH_MAPPED_GENES"] = prev_associated_traits

Then, I merged the two datasets.

In [8]:
# Add matching columns before merging them
all_traits["BETA"] = all_traits["Beta"]
all_traits["PVALUE"] = all_traits["P-value"]
all_traits["TRAIT"] = all_traits["Trait(s)"]
all_traits_subset = all_traits[["BETA", "CHR", "BP", "OR", "RAF", "PVALUE", "VARIANT", "EFFECT_ALLELE", "TRAIT", 
                                "MAPPED_GENE_LIST", "NUM_MAPPED_GENES", "TRAITS_ASSOCIATED_WITH_MAPPED_GENES"]]

all_traits_merged = all_traits_subset.append(scraped_df, ignore_index = True)

### Data Cleaning ###

In [9]:
# Remove the NA BETA rows (only 20) in order to convert the strings into ints
all_traits_merged = all_traits_merged[all_traits_merged["BETA"].notna()]

new_beta_list = []
for val in all_traits_merged["BETA"]:
    sub_list = val.split(" ")
    if (len(sub_list) == 1):
        new_beta_list.append(-10) # Use -10 to represent NAN -- will not process these values
    elif (sub_list[2] == "increase"):
        new_beta_list.append(float(sub_list[0])) # leave increases as positive values
    else:
        new_beta_list.append(-1 * float(sub_list[0])) # add a negative to represent a decrease
all_traits_merged["BETA"] = new_beta_list

new_raf_list = []
for val in all_traits_merged["RAF"]:
    if type(val) == float or type(val) == int:
        new_raf_list.append(val)
    elif val.isnumeric():
        new_raf_list.append(float(val))
    else:
        new_raf_list.append(-10)
all_traits_merged["RAF"] = new_raf_list

new_pvalue_list = []
for val in all_traits_merged["PVALUE"]:
    temp = val.split(" ")
    first = float(temp[0])
    exponent = float(temp[2].split("-")[1])
    out = first * 10 ** (-1 * exponent)
    new_pvalue_list.append(out)
all_traits_merged["PVALUE"] = new_pvalue_list

In [10]:
# Convert other values to numbers, if NA set to -10
all_traits_merged["CHR"] = [ float(i) if i.isnumeric() else -10 for i in all_traits_merged["CHR"] ]
all_traits_merged["BP"] = [ float(i) if i.isnumeric() else -10 for i in all_traits_merged["BP"] ]
all_traits_merged["OR"] = [ float(i) if i != "'-" else -10 for i in all_traits_merged["OR"] ]

In [11]:
all_traits_merged = all_traits_merged[ all_traits_merged["EFFECT_ALLELE"] != "D" ]
all_traits_merged.head()

,BETA,CHR,BP,OR,RAF,PVALUE,VARIANT,EFFECT_ALLELE,TRAIT,MAPPED_GENE_LIST,NUM_MAPPED_GENES,TRAITS_ASSOCIATED_WITH_MAPPED_GENES
0,-10.0,11.0,76588150.0,1.22,-10.0,1.000000e-08,rs2155219,T,seasonal allergic rhinitis,"[EMSY, AP001189.2]",2,-
1,-10.0,5.0,110810746.0,1.39,-10.0,1.000000e-08,rs17513503,G,seasonal allergic rhinitis,"[SLC25A46, AC008782.1]",2,-
2,-10.0,4.0,122451978.0,1.19,-10.0,1.000000e-06,rs2069772,C,seasonal allergic rhinitis,[IL2],1,-
3,-10.0,16.0,9305867.0,1.18,-10.0,2.000000e-06,rs631208,A,seasonal allergic rhinitis,"[LINC02177, RPL21P119]",2,-
4,-10.0,1.0,51393570.0,1.35,-10.0,2.000000e-06,rs6673480,T,seasonal allergic rhinitis,[EPS15],1,-


Finally, I added a new column with discrete base pair locations based on total number of base pairs in that chromosome. I also added a new column with discrete values for the effect allele. [Here](https://www.ncbi.nlm.nih.gov/books/NBK22266/) is the source for my total base pair dictionary.

In [35]:
# Map chromosome to total number of base pairs
total_base_pair = {1: 240e6, 2: 240e6, 3: 200e6, 4: 190e6, 5: 180e6, 6: 170e6, 7: 150e6, 8: 140e6, 9: 130e6, 
                   10: 130e6, 11: 130e6, 12: 130e6, 13: 110e6, 14: 100e6, 15: 100e6, 16: 90e6, 17: 80e6,
                   18: 70e6, 19: 60e6, 20: 60e6, 21: 40e6, 22: 40e6}

new_bp_list = []
for num in range(all_traits_merged.shape[0]):
    row = all_traits_merged.iloc[num]
    chrom = row["CHR"]
    bp = row["BP"]
    
    if (chrom == -10 or bp == -10):
        new_bp_list.append(-10)
    elif (bp < (0.25 * total_base_pair[chrom])):
        new_bp_list.append(1)
    elif (bp < (0.50 * total_base_pair[chrom]) and (bp >= (0.25 * total_base_pair[chrom]))):
        new_bp_list.append(2)
    elif (bp < (0.75 * total_base_pair[chrom]) and (bp >= (0.50 * total_base_pair[chrom]))):
        new_bp_list.append(3)
    else:
        new_bp_list.append(4)

all_traits_merged["BP_DISCRETE"] = new_bp_list

In [37]:
effect_allele = {"T": 1, "A": 2, "G": 3, "C": 4, "?": -10}

new_effect_allele_list = []
for val in all_traits_merged["EFFECT_ALLELE"]:
    new_effect_allele_list.append(effect_allele[val])

all_traits_merged["EFFECT_ALLELE_DISCRETE"] = new_effect_allele_list

In [38]:
all_traits_merged.head()

,BETA,CHR,BP,OR,RAF,PVALUE,VARIANT,EFFECT_ALLELE,TRAIT,MAPPED_GENE_LIST,NUM_MAPPED_GENES,TRAITS_ASSOCIATED_WITH_MAPPED_GENES,BP_DISCRETE,EFFECT_ALLELE_DISCRETE
0,-10.0,11.0,76588150.0,1.22,-10.0,1.000000e-08,rs2155219,T,seasonal allergic rhinitis,"[EMSY, AP001189.2]",2,-,3,1
1,-10.0,5.0,110810746.0,1.39,-10.0,1.000000e-08,rs17513503,G,seasonal allergic rhinitis,"[SLC25A46, AC008782.1]",2,-,3,3
2,-10.0,4.0,122451978.0,1.19,-10.0,1.000000e-06,rs2069772,C,seasonal allergic rhinitis,[IL2],1,-,3,4
3,-10.0,16.0,9305867.0,1.18,-10.0,2.000000e-06,rs631208,A,seasonal allergic rhinitis,"[LINC02177, RPL21P119]",2,-,1,2
4,-10.0,1.0,51393570.0,1.35,-10.0,2.000000e-06,rs6673480,T,seasonal allergic rhinitis,[EPS15],1,-,1,1


In [12]:
# Save final dataset as a CSV
all_traits_merged.to_csv("final_dataset.csv", index = False)